# Tutorial For GFETM

In [ ]:
# import the necessary packages for processing
import anndata
import pysam
import numpy as np
import os
import umap
import matplotlib.pyplot as plt

Load the necessary dataset with anndata

Download the fasta file from here: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz 

Remember different datasets may have different reference genomes, so it would be better to check at the scATAC-seq dataset decription before you start

In [ ]:
adata = anndata.read_h5ad("../data/buen18.h5ad")
fasta_file = 'hg19.fa'
fasta_open = pysam.Fastafile(fasta_file)


Extract the peak sequences and save to txt file

In [ ]:
chrom = np.array(adata.var['chr'])
start = np.array(adata.var['start'])
end = np.array(adata.var['end'])
seq_dna = []
for i in range(len(chrom)):
    seq_dna.append(fasta_open.fetch(chrom[i], int(start[i]), int(end[i])).upper())
with open("peak_sequences_buen18.txt","w") as f:
    for i in seq_dna:
        f.writelines(i+"\n")

Download the DNABERT pre-trained model from https://drive.google.com/file/d/1BJjqb5Dl2lNMg2warsFQ0-Xvn1xxfFXC/view?usp=sharing

Unzip the DNABERT pre-trained model

Use transformers to tokenize DNA peak sequences to tokens and save to h5 file

In [ ]:
import h5py
import anndata
from transformers import AutoTokenizer, AutoModel
import torch 
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("../gfm_checkpoint/6-new-12w-0")
kmer = 6
count = 0
device = "cuda"
mean_embedding = []
cls_embedding = []
with open("peak_sequences_buen18.txt") as f:
    dna_array_dense = []
    for line in f:
        print(count)
        line = line.rstrip()
        count += 1
        if len(line) > 512:
            gap = int((len(line) - 512) //2) + 1
            line = line[gap:-gap]
        processed = ""
        for q in range(509):
            processed +=  line[q:q+kmer] +" "
        dna_array_dense.append(tokenizer.encode_plus(processed,padding="max_length")['input_ids'])

dna_array_dense = np.array(dna_array_dense)
adata = anndata.AnnData(dna_array_dense)
adata.write("../data/buen18_seq.h5")

Run the code for training the GFETM model

In [ ]:
! python main.py --mode train  --data_path ../data/buen18.h5ad --seq_path ../data/buen18_seq.h5 --checkpoint_path ../gfm_checkpoint/6-new-12w-0 --enc_drop 0.1 --num_topics 24 --seed 4 --epochs 4000 --emb_size 768 --rho_size 768 --t_hidden_size 1000 --output_path ../outputs/

Analyze the Results

In [ ]:
adata = anndata.read_h5ad("../outputs/adata.h5ad")
cell_embeddings = adata.obsm['projection']

In [ ]:
umap_obj = umap.UMAP()
umap_coords = umap_obj.fit_transform(cell_embeddings)
adata.obsm['X_umap'] = umap_coords

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sc.pl.embedding(adata, 'X_umap', color="label",ax=ax,legend_fontsize="xx-small",frameon=False)
ax.set_title("")
plt.tight_layout()
plt.savefig("embeddings_color_by_cell_type.png")

fig, ax = plt.subplots(figsize=(7, 7))
sc.pl.embedding(adata, 'X_umap', color="batch",ax=ax,legend_fontsize="xx-small",frameon=False)
ax.set_title("")
plt.tight_layout()
plt.savefig("embeddings_color_by_batch.png")